In [2]:
import os, sys
sys.path.append("../../../")

from src.core.module import Module
from src.core.losses import CrossEntropy
from src.core.optim import AdamW
from src.core.tensor import Tensor
from src.utils.backend import xp

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np




In [ ]:
src = np.random.randint(low=1, high=16, size=(128, 16))
x = src[:, :-1]
y = src[:, 1:]

x_mine = Tensor(x, requires_grad=False)
y_mine = Tensor(y, requires_grad=False)


class Net(Module):
    def __init__(self, d_model, n_heads, vocab_size, max_seq_len, pad_idx=0):
        super().__init__()
        self.e = self.embedding(vocab_size, d_model, max_seq_len, pad_idx, name="Embedding")

        self.head1 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head2 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head3 = self.transformer(d_model=d_model, n_heads=n_heads)
        # self.head4 = self.transformer(d_model=d_model, n_heads=n_heads)
        # self.head5 = self.transformer(d_model=d_model, n_heads=n_heads)
        # self.head6 = self.transformer(d_model=d_model, n_heads=n_heads)
        # self.head7 = self.transformer(d_model=d_model, n_heads=n_heads)
        # self.head8 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.project = self.linear(d_model, vocab_size, name="project")
    
    def forward(self, idx):
        x, padding_mask = self.e.get_sentence_embedding(idx)
        x = Tensor(x.data, requires_grad=False)
        x = self.head1(x, padding_mask)
        x = self.head2(x, padding_mask)
        x = self.head3(x, padding_mask)
        # x = self.head4(x, padding_mask)
        # x = self.head5(x, padding_mask)
        # x = self.head6(x, padding_mask)
        # x = self.head7(x, padding_mask)
        # x = self.head8(x, padding_mask)
        x = self.project(x)
        return x

    def train(self, x, y, epochs, optimizer):
        for epoch in range(epochs):
            y_hat = self.forward(x)
            # print(y_hat.shape, y.shape)
            loss = CrossEntropy(y_hat, y, axis=-1)
    
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if epoch % 1 == 0:
                print(f"Epoch {epoch}, Loss: {loss.data}")
                
if __name__ == "__main__":
    D_MODEL = 48
    VOCAB_SIZE = 20
    N_HEADS = 2
    MAX_SEQ_LEN = 32
    PAD_IDX = 0

    model = Net(d_model=D_MODEL, n_heads=N_HEADS, vocab_size=VOCAB_SIZE, max_seq_len=MAX_SEQ_LEN, pad_idx=PAD_IDX)
    model._build((128, 15))
    optimizer = AdamW(model.parameters(), lr=0.001, precision=(xp.float32, xp.float32))


    model.train(x_mine, y_mine, epochs=1000, optimizer=optimizer)


    
        

Epoch 0, Loss: 5.8359375
Epoch 1, Loss: 5.265625
Epoch 2, Loss: 4.8515625
Epoch 3, Loss: 4.578125
Epoch 4, Loss: 4.375
Epoch 5, Loss: 4.1953125
Epoch 6, Loss: 4.015625
Epoch 7, Loss: 3.84765625
Epoch 8, Loss: 3.70703125
Epoch 9, Loss: 3.595703125
Epoch 10, Loss: 3.505859375
Epoch 11, Loss: 3.423828125
Epoch 12, Loss: 3.34375
Epoch 13, Loss: 3.263671875
Epoch 14, Loss: 3.1875
Epoch 15, Loss: 3.12109375
Epoch 16, Loss: 3.0703125
Epoch 17, Loss: 3.029296875
Epoch 18, Loss: 2.994140625
Epoch 19, Loss: 2.955078125
Epoch 20, Loss: 2.9140625
Epoch 21, Loss: 2.873046875
Epoch 22, Loss: 2.837890625


KeyboardInterrupt: 